# Install dan Import Library

In [1]:
# Install required packages
!pip install google-play-scraper
!pip install nltk
!pip install Sastrawi
!pip install wordcloud
!pip install seaborn
!pip install tensorflow
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Text processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D, GRU, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Mengambil data ulasan

In [3]:
from google_play_scraper import Sort, reviews

# Scrape reviews
scrapreview, continuation_token = reviews(
    'com.HoYoverse.hkrpgoversea',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=5000,  # Increased sample size
    filter_score_with=None
)

# Load data dari hasil scraping yang sudah disimpan
df_review = pd.read_csv('https://raw.githubusercontent.com/Alfan345/Analisis-sentimen-game-Honkai-Star-Rail/refs/heads/main/ulasan_aplikasi.csv')  # pastikan file ini tersedia
df_review = pd.DataFrame(scrapreview)
df_review.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,32063696-a983-49ac-8378-8b99cda18013,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"cut scene, character, bangunan, grafik,alur ce...",5,178,3.2.0,2025-04-09 11:21:03,None,NaT,3.2.0
1,21eaa8ba-36ef-4441-97ea-112607515be0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Penulisan ceritanya cukup bagus bisa ditingkat...,5,3,3.2.0,2025-04-16 11:56:51,None,NaT,3.2.0
2,46fa15e9-6c03-4048-add2-8f47fe55caf7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Banyak kemunduran dalam perkembangan game ini,...",1,26,3.2.0,2025-04-15 06:23:45,None,NaT,3.2.0
3,ebaa30c5-ddc7-448b-b4e3-34dacecaf748,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Visual oke, gameplay mudah dipahami, story lum...",4,7,3.2.0,2025-04-09 03:20:15,None,NaT,3.2.0
4,11e55296-1306-4ca8-b365-c58b6ddc5f73,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"saya sangat suka dengan game ini , semenjak up...",5,28,3.2.0,2025-04-12 13:57:43,None,NaT,3.2.0


# Pre-processing data

In [4]:
import string, requests, json
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')



# Labeling based on score
def label_score(score):
    if score < 3:
        return 'Negatif'
    elif score == 3:
        return 'Netral'
    elif score >= 4:
        return 'Positif'

my_df = df_review[['userName', 'score', 'at', 'content']]
my_df['Label'] = my_df['score'].apply(label_score)

# Check class distribution
print(my_df['Label'].value_counts())

# Balance the dataset
from sklearn.utils import resample
df_neg = my_df[my_df['Label'] == 'Negatif']
df_net = my_df[my_df['Label'] == 'Netral']
df_pos = my_df[my_df['Label'] == 'Positif']

# Upsample minority classes
df_neg_upsampled = resample(df_neg, replace=True, n_samples=len(df_pos), random_state=42)
df_net_upsampled = resample(df_net, replace=True, n_samples=len(df_pos), random_state=42)

# Combine majority class with upsampled minority classes
my_df = pd.concat([df_pos, df_neg_upsampled, df_net_upsampled])

# Ambil slangword
get_slangs = requests.get('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/refs/heads/master/combined_slang_words.txt')
slangwords = json.loads(get_slangs.text) if get_slangs.status_code == 200 else {}

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip()

def casefoldingText(text):
    return text.lower()

def tokenizingText(text):
    # Daripada word_tokenize (yang butuh punkt), gunakan split
    return text.split()


def lemmatingText(text):
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(word, pos='v') for word in text])

def fix_slangwords(text):
    return ' '.join([slangwords.get(word, word) for word in text.split()])

def filteringText(words):
    factory = StopWordRemoverFactory()
    stopwords = set(factory.get_stop_words())
    return [word for word in words if word not in stopwords]

def toSentence(word_list):
    return ' '.join(word_list)

my_df['processed'] = my_df['content'].astype(str).apply(cleaningText)
my_df['processed'] = my_df['processed'].apply(casefoldingText)
my_df['processed'] = my_df['processed'].apply(fix_slangwords)
my_df['processed'] = my_df['processed'].apply(tokenizingText)    # ← pakai split sekarang
my_df['processed'] = my_df['processed'].apply(filteringText)
my_df['processed'] = my_df['processed'].apply(lemmatingText)

# Terakhir, stemming
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()
my_df['content_stemmed'] = my_df['processed'].apply(lambda x: stemmer.stem(x))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-4-f183d2b4be95>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['Label'] = my_df['score'].apply(label_score)


Label
Positif    3189
Negatif    1392
Netral      419
Name: count, dtype: int64


In [14]:
import os, random
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

In [15]:
# Split data
X = my_df['content_stemmed']
y = my_df['Label']

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Pelatihan Model

In [16]:
# Create a pipeline with TF-IDF and classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=10000,
        ngram_range=(1, 2),  # Use unigrams and bigrams
        min_df=5,
        max_df=0.7
    )),
    ('clf', GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        random_state=42
    ))
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
print("Gradient Boosting Classifier:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Gradient Boosting Classifier:
              precision    recall  f1-score   support

     Negatif       0.86      0.94      0.90       638
      Netral       0.94      0.99      0.97       638
     Positif       0.94      0.81      0.87       638

    accuracy                           0.91      1914
   macro avg       0.91      0.91      0.91      1914
weighted avg       0.91      0.91      0.91      1914

Accuracy: 0.9107


In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Tokenizer dan padding ulang
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

max_len = 200
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')


# Model GRU
model_gru = Sequential([
    Embedding(input_dim=20000, output_dim=128),
    SpatialDropout1D(0.3),
    Bidirectional(GRU(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_gru.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint(
    'best_gru_model.h5', monitor='val_accuracy',
    save_best_only=True, mode='max', verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy', patience=5,
    mode='max', restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.5,
    patience=2, min_lr=1e-5, verbose=1, mode='max'
)

history = model_gru.fit(
    X_train_pad, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

# Evaluasi
model_gru.load_weights('best_gru_model.h5')

y_pred = model_gru.predict(X_test_pad)
y_pred = np.argmax(y_pred, axis=1)

print("\n📊 GRU Model Evaluation:")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3498 - loss: 1.0970
Epoch 1: val_accuracy improved from -inf to 0.47911, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - accuracy: 0.3501 - loss: 1.0970 - val_accuracy: 0.4791 - val_loss: 1.0858 - learning_rate: 3.0000e-04
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4353 - loss: 1.0817
Epoch 2: val_accuracy improved from 0.47911 to 0.58486, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.4357 - loss: 1.0815 - val_accuracy: 0.5849 - val_loss: 1.0302 - learning_rate: 3.0000e-04
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5613 - loss: 0.9841
Epoch 3: val_accuracy improved from 0.58486 to 0.62794, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.5619 - loss: 0.9831 - val_accuracy: 0.6279 - val_loss: 0.8094 - learning_rate: 3.0000e-04
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6649 - loss: 0.7668
Epoch 4: val_accuracy improved from 0.62794 to 0.76893, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 143s 3s/step - accuracy: 0.6656 - loss: 0.7660 - val_accuracy: 0.7689 - val_loss: 0.6306 - learning_rate: 3.0000e-04
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7814 - loss: 0.5776
Epoch 5: val_accuracy improved from 0.76893 to 0.81462, saving model to best_gru_model.h5


...

54/54 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9098 - loss: 0.2963 - val_accuracy: 0.8930 - val_loss: 0.3356 - learning_rate: 3.0000e-04
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9261 - loss: 0.2294
Epoch 8: val_accuracy improved from 0.89295 to 0.89948, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.9263 - loss: 0.2290 - val_accuracy: 0.8995 - val_loss: 0.3216 - learning_rate: 3.0000e-04
Epoch 9/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9479 - loss: 0.1683
Epoch 9: val_accuracy improved from 0.89948 to 0.90339, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.9479 - loss: 0.1682 - val_accuracy: 0.9034 - val_loss: 0.3192 - learning_rate: 3.0000e-04
Epoch 10/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9508 - loss: 0.1623
Epoch 10: val_accuracy did not improve from 0.90339
54/54 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9509 - loss: 0.1620 - val_accuracy: 0.9008 - val_loss: 0.3336 - learning_rate: 3.0000e-04
Epoch 11/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9596 - loss: 0.1294
Epoch 11: val_accuracy improved from 0.90339 to 0.91253, saving model to best_gru_model.h5


54/54 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9597 - loss: 0.1292 - val_accuracy: 0.9125 - val_loss: 0.3024 - learning_rate: 3.0000e-04
Epoch 12/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9643 - loss: 0.1127
Epoch 12: val_accuracy did not improve from 0.91253
54/54 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - accuracy: 0.9644 - loss: 0.1124 - val_accuracy: 0.9060 - val_loss: 0.3529 - learning_rate: 3.0000e-04
Epoch 13/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9714 - loss: 0.0939
Epoch 13: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.

Epoch 13: val_accuracy did not improve from 0.91253
54/54 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.9715 - loss: 0.0938 - val_accuracy: 0.9047 - val_loss: 0.3442 - learning_rate: 3.0000e-04
Epoch 14/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9779 - loss: 0.0757
Epoch 14: val_accuracy did not improve from 0.91253
54/54 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9779 - loss: 0.0757 - val_ac

54/54 ━━━━━━━━━━━━━━━━━━━━ 202s 3s/step - accuracy: 0.9785 - loss: 0.0718 - val_accuracy: 0.9178 - val_loss: 0.3306 - learning_rate: 1.5000e-04
Epoch 16/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9795 - loss: 0.0702
Epoch 16: val_accuracy did not improve from 0.91775
54/54 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.9796 - loss: 0.0701 - val_accuracy: 0.9125 - val_loss: 0.3399 - learning_rate: 1.5000e-04
Epoch 17/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9818 - loss: 0.0612
Epoch 17: ReduceLROnPlateau reducing learning rate to 7.500000356230885e-05.

Epoch 17: val_accuracy did not improve from 0.91775
54/54 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.9818 - loss: 0.0612 - val_accuracy: 0.9151 - val_loss: 0.3373 - learning_rate: 1.5000e-04
Epoch 18/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9826 - loss: 0.0609
Epoch 18: val_accuracy did not improve from 0.91775
54/54 ━━━━━━━━━━━━━━━━━━━━ 148s 3s/step - accuracy: 0.9827 - loss: 0.0608 - val_ac

# Inference

In [20]:
def preprocess_for_inference(text):
    text = cleaningText(text)
    text = fix_slangwords(text)
    tokens = filteringText(tokenizingText(text))
    text = lemmatingText(toSentence(tokens))
    text = stemmer.stem(text)
    return text

def predict_sentiment_gru(text):
    cleaned = preprocess_for_inference(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    pred_idx = np.argmax(model_gru.predict(pad), axis=1)[0]
    return encoder.classes_[pred_idx]

inference_samples = [
    "game ini bikin nagih banget, seru parah!",
    "fitur-fiturnya terlalu ribet dan berat",
    "grafik dan animasinya oke, tapi loading lama",
    "biasa aja sih, gak spesial",
    "sangat membantu untuk mengisi waktu luang",
    "game nya jelek banget, uninstall langsung",
    "Performanya belum di optimalisasikan disemua perangkat"
]

print("\n Inference dengan GRU Model:")
for i, sample in enumerate(inference_samples, 1):
    pred = predict_sentiment_gru(sample)
    print(f"{i}. \"{sample}\" → Prediksi: {pred}")



 Inference dengan GRU Model:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1. "game ini bikin nagih banget, seru parah!" → Prediksi: Positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
2. "fitur-fiturnya terlalu ribet dan berat" → Prediksi: Positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
3. "grafik dan animasinya oke, tapi loading lama" → Prediksi: Negatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
4. "biasa aja sih, gak spesial" → Prediksi: Positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
5. "sangat membantu untuk mengisi waktu luang" → Prediksi: Positif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
6. "game nya jelek banget, uninstall langsung" → Prediksi: Negatif
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
7. "Performanya belum di optimalisasikan disemua perangkat" → Prediksi: Positif
